# Agent Learning Project

**This code is not yet functional. I am currently in the process of implementing it. I will provide with updates as soon as I can.**

## Imports 

We start by importing the required libaries. Uncomment & run the next cell in case some of the libraries aren't installed in your environment. 

In [1]:
# !pip install numpy matplotlib
# !pip install gymnasium

In [2]:
# I have to run this because I have a kernel problem but you shouldn't have to run it
run = True
if run:
    import sys
    sys.path.append('/opt/homebrew/lib/python3.11/site-packages')

In [3]:
import numpy as np # Useful for computations and probabilistic distributions
import matplotlib.pyplot as plt # Plotting Graph for Simulation results
from collections import defaultdict # Specific kind of dictionary that avoids KeyErrors
import gymnasium as gym
from gymnasium import spaces

## Defining the Agent class

We define a type of agent with the following attributes: 
- Agent name : Name of the agent
- Service probability : If at the counter, shows the probability that this agent will be serviced.
- Reward : Reward the counter gets for servicing this agent
- Time in queue : Time for which the agent has been in the queue. 

In [4]:
class Agent:
    def __init__(self, agent_name):
        self.agent_name = agent_name
        self.service_prob = agents[agent_name][0]
        self.reward = agents[agent_name][1]
        self.penalty = agents[agent_name][2]
        self.time_in_queue = 0

    def __str__(self):
        return f"Agent {self.agent_name} ({self.time_in_queue})"

## Defining the Queue mechanism 

In [5]:
class QueueSimulation:
    def __init__(self, agents, arrival_prob, strategy, init, alpha, discount):
        # Define the agents, their distribution, and the strategy
        self.agents = agents
        self.agent_distribution = np.array([agent[3] for agent in agents.values()])
        self.agent_distribution = self.agent_distribution / np.sum(self.agent_distribution)
        self.strategy = strategy
        
        # Define the hyper-parameters of the simulation
        self.arrival_prob, self.init, self.alpha, self.discount = arrival_prob, init, alpha, discount
        self.fixed_discount = self.discount # Optimization trick to avoid recomputing power at each step
        
        # Define the Queue
        self.queue = [self.create_agent() for _ in range(init)] if type(init) == int else init
        self.current_agent = None if len(self.queue) == 0 else self.queue[0]
        
        # Variables that are used to analyze our code
        self.time, self.busy, self.total_reward, self.discounted_reward = 0, 0, 0, 0
        
        # Tracking the history for graphs
        self.serviced_agents = defaultdict(int)
        self.reward_history, self.queue_length_history = [], []

    def create_agent(self):
        agent_name = np.random.choice(list(self.agents.keys()), p=self.agent_distribution)
        return Agent(agent_name)

    def step(self):
        """
        This function is the core of the simulation. It defines what happens at every step. The order in which 
        the computations are made is important so we have to be careful. In our case, we proceed as follows:
        
        1. Determine whether an agent is joining the queue
        2. Detemine if the agent in service is going to complete service or not and potentially add reward 
        3. Apply the given strategy to determine what happens next
        4. Add penalty to the reward
        5. Add the information related to the step to the simulation history
        
        Please feel free to ask for any changes in the implementation
        """
        self.time += 1 # Update time
        
        # Add new agent to the queue based on arrival probability
        if np.random.rand() < self.arrival_prob:
            new_agent = self.create_agent()
            self.queue.append(new_agent)

        # Check if the current agent is done being served
        if self.current_agent:
            if np.random.rand() < self.current_agent.service_prob:
                self.total_reward += self.current_agent.reward
                self.discounted_reward += self.current_agent.reward*self.discount
                self.serviced_agents[self.current_agent.agent_name] += 1
                self.current_agent = None
                
        # Apply the strategy to select the next agent
        if self.queue:
            queue_index = self.strategy(self.queue,self.current_agent)
            self.swap_agents(queue_index)
        
        length = len(self.queue)
        
        # Apply penalty on reward
        for agent in self.queue: 
            agent.time_in_queue += 1
            self.total_reward -= agent.penalty
            self.discounted_reward -= agent.penalty*self.discount
        self.total_reward -= regularization(self.alpha,length)
        self.discounted_reward -= regularization(self.alpha,length)*self.discount
        
        # Add the step's information to the simulation history
        self.reward_history.append(self.total_reward)
        self.queue_length_history.append(length+bool(self.current_agent))
        self.busy += bool(self.current_agent)
        
        self.discount *= self.discount # Update Discount

    def swap_agents(self, queue_index):
        if 0 <= queue_index < len(self.queue):
            if self.current_agent:
                self.queue[queue_index], self.current_agent = (self.current_agent,self.queue[queue_index])
            else:
                self.current_agent = self.queue.pop(queue_index)

    def run_simulation(self, steps=1000):
        for _ in range(steps):
            self.step()

    def get_results(self):
        # Plot total reward over time
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.plot(self.reward_history)
        plt.title('Total Reward over Time')
        plt.xlabel('Time')
        plt.ylabel('Total Reward')

        # Plot queue length over time
        plt.subplot(1, 3, 2)
        plt.plot(self.queue_length_history)
        plt.title('Queue Length over Time')
        plt.xlabel('Time')
        plt.ylabel('Queue Length')

        # Plot distribution of serviced agents by type
        plt.subplot(1, 3, 3)
        agent_types, counts = zip(*sorted(self.serviced_agents.items()))
        plt.bar(agent_types, counts)
        plt.title('Distribution of Serviced Agents by Type')
        plt.xlabel('Agent Type')
        plt.ylabel('Count')

        plt.tight_layout()
        plt.show()
        
        self.min_queue_length = np.min(self.queue_length_history)
        self.max_queue_length = np.max(self.queue_length_history)
        self.mean_queue_length = np.mean(self.queue_length_history)
        self.normalized_distribution = {agent_type: round(count / sum(self.serviced_agents.values()), 3) for agent_type, count in sorted(self.serviced_agents.items(), key=lambda item: item[1], reverse=True)}

        
        print(f"\nTotal reward: {self.total_reward}")
        print(f"Discounted reward: {self.discounted_reward}")
        print(f"Average reward per step: {self.total_reward/self.time}")
        print(f"Min & Max Queue Length: {self.min_queue_length} & {self.max_queue_length}")
        print(f"Mean Queue Length: {self.mean_queue_length}")
        print(f"Normalized Distribution of Serviced Agents: {self.normalized_distribution}")
        print(f"Time spent busy: {self.busy/self.time}")
        print(f"--------------------------------------------------------")


In [6]:
# Compute average service probability
def service():
    service_prob = np.array([agent[0] for agent in agents.values()])
    weights = np.array([agent[3] for agent in agents.values()])
    distribution = weights / np.sum(weights)
    x = np.dot(service_prob, distribution.T)
    return x,1/x

# Compute expect reward per agent
def average():
    probabilities = np.array([agent[0] for agent in agents.values()])
    rewards = np.array([agent[1] for agent in agents.values()])
    penalties = np.array([agent[2] for agent in agents.values()])
    average_values = {agent: (rewards[i] + penalties[i]) / probabilities[i] for i, agent in enumerate(agents.keys())}
    return average_values

# Create the initial list
def create(l, shuffle=False):
    queue = []
    for name, quantity in l:
        queue += [Agent(name)] * quantity
    if shuffle:
        np.random.shuffle(queue)
    return queue

## Choosing base parameters

In [7]:
# Defining Agents
agents = {
    ### Agent name : [service_probability, reward, penalty, distrib_weight]
    # service_probability: float in [0,1]
    # reward, penalty, distrib_weight: float (unconstrained)
    "Alex" : [0.7, 1, 0.1, 6], 
    "Ben" : [0.5, 20, 0.4, 3],
    "Cameron" : [0.3, 30, 0.2, 4],
    "Dennis" : [0.4, 20, 0.01, 2],
    "Eric": [0.8, 4, 0.2, 3],
    "Fabien": [0.5, 50, 0.4, 1],
}

# Probabilty that a new agent joins the queue
arrival_prob = 0.5

# Number of agents in the queue at t = 0
# We either choose a random weighted queue or a specific queue
start = 0
queue = create([("Fabien", 20), ("Alex", 10)], True) # [(name, quantity),(name, quantity),...], shuffle=True/False 
init = start # Choose a mode here

# Choose a discount factor for discounted reward computation
discount = 0.99

# Regularization parameter and function
alpha = 0.00
def regularization(alpha,length):
    return alpha*length

## Running the Q-Learning model

Execute the main code automatically by choosing one of the strategies defined above in the 'Define Strategies' section and letting the computer play by itself. This function outputs three graphs. 
- Graph 1: Total reward from time $t_0$ to time $t_{1000}$.
- Graph 2: Length of the Queue from time $t_0$ to time $t_{1000}$.
- Graph 3: Distribution of serviced Agents by type from $t_0$ to time $t_{1000}$.



In [8]:
simulation = QueueSimulation(
    agents,
    arrival_prob,
    strategy,
    init,
    alpha,
    discount,
)

print(f"Automatic simulation with strategy '{strategy.__name__}'")
print(f"Expected service probability & time across agents: {service()}")
print(f"Expected reward + weighted penalty for each agent: {average()}")
simulation.run_simulation(steps=10000)
simulation.get_results()

NameError: name 'strategy' is not defined

# Comments

In case something needs to be changed, please feel free to write something in the cell below and email me about it. 